In [ ]:
from knock72 import *  # 必要な関数やクラスをインポートしていると仮定
from torch.utils.data import Dataset  # Datasetクラスをtorch.utils.dataからインポート
from torch.utils.data import DataLoader  # DataLoaderクラスをtorch.utils.dataからインポート
from matplotlib import pyplot as plt  # matplotlibのpyplotをインポート
import numpy as np  # numpyをインポート
import time  # 時間計測のためにtimeモジュールをインポート

# モデルをトレーニングする関数
def train_model(dataset_train, dataset_valid, batch_size, model, criterion, num_epochs):
    # トレーニング用データローダーの作成
    dataloader_train = DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True)
    # バリデーション用データローダーの作成
    dataloader_valid = DataLoader(
        dataset_valid, batch_size=len(dataset_valid), shuffle=False)

    log_train = []  # トレーニングデータのログを保存するリスト
    log_valid = []  # バリデーションデータのログを保存するリスト

    for epoch in range(num_epochs):  # エポックの数だけ繰り返す
        start_time = time.time()  # エポックの開始時間を記録

        model.train()  # モデルをトレーニングモードに設定
        loss_train = 0.0  # トレーニングロスの初期化
        for inputs, labels in dataloader_train:  # トレーニングデータローダーからバッチを取得
            optimizer.zero_grad()  # オプティマイザの勾配をゼロにリセット

            outputs = model(inputs)  # モデルに入力を与えて出力を取得
            loss = criterion(outputs, labels)  # 出力とラベルからロスを計算
            loss.backward()  # 誤差逆伝播を実行
            optimizer.step()  # パラメータの更新
        end_time = time.time()  # エポックの終了時間を記録

        # トレーニングデータに対するロスと精度を計算
        loss_train, acc_train = calc_loss_acc(
            model, criterion, dataloader_train)
        # バリデーションデータに対するロスと精度を計算
        loss_valid, acc_valid = calc_loss_acc(
            model, criterion, dataloader_valid)
        log_train.append([loss_train, acc_train])  # トレーニングログに追加
        log_valid.append([loss_valid, acc_valid])  # バリデーションログに追加

        # モデルとオプティマイザの状態を保存
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(
        ), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

        # エポックごとの結果を出力
        print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, train_time: {(end_time - start_time):.4f}sec')

# モデルの定義と設定
model = SingleLayerPerceptronNetwork(300, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
num_epochs = 1

# バッチサイズを変更しながらモデルをトレーニング
for batch_size in [2 ** i for i in range(12)]:
    print(f"batch_size : {batch_size}")
    train_model(dataset_train, dataset_valid,
                batch_size, model, criterion, num_epochs)

"""
結果の例:
batch_size : 1
epoch: 1, loss_train: 0.3263, accuracy_train: 0.8896, loss_valid: 0.3279, accuracy_valid: 0.8892, train_time: 0.4831sec
batch_size : 2
epoch: 1, loss_train: 0.3013, accuracy_train: 0.8969, loss_valid: 0.3120, accuracy_valid: 0.8952, train_time: 0.3185sec
...
batch_size : 2048
epoch: 1, loss_train: 0.2861
